In [ ]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

# Answer Removed ITEM Level Processing


In [ ]:
self = features_class
feature_name = 'f__answer_removed'
score_name = 's__answer_changed'
df = self.get_feature_item__answer_removed(feature_name)
df = df[~pd.isnull(df[feature_name])].copy()

In [ ]:
variables = list(df.variable_name.unique())

In [ ]:
from pyod.models.ecod import ECOD
model = ECOD(contamination=0.1)

df[score_name] = None
df['value'] = None
for index_range in range(0, len(variables)):
    mask = (df['variable_name'] == variables[index_range])
    #df.loc[mask,score_name] = model.fit_predict(df[mask][[feature_name]])
    df.loc[mask,'value'] = df[mask][feature_name].apply(lambda x: 5 if x>5 else x)

In [ ]:
for var in variables:
    mask = (df['variable_name'] == var)

    model = ECOD(contamination=0.11)
    model.fit(df[mask][[feature_name]])
    df.loc[mask, score_name] = model.predict(df[mask][[feature_name]])

In [ ]:
print(df[score_name].value_counts(), df[score_name].value_counts()/df[score_name].count())

In [ ]:
mask = df['value']>0
for index_range in [x for x in range(0, len(variables), 50)]:
    plt.figure(figsize=(15, 6))
    sns.boxplot(df[(df['variable_name'].isin(variables[index_range:index_range+50]))&mask],x='variable_name', y='value', hue=score_name)
    plt.xticks(rotation=90)
    plt.show()

# Answer Removed UNIT Level Processing


In [ ]:
# Plot the anomalies in answer_changed by interview__id  in the calculation it fill by zero empty values as it would imply "no anmalies detected"
data = df.groupby(['interview__id'])[score_name].mean()
data = data.reset_index()
data[score_name].hist()

In [ ]:
sns.boxplot(data,y=score_name)
plt.show()

In [ ]:
total_unit = data['interview__id'].count()
mean_value = data[score_name].mean()
print(f" Total UNITS: {total_unit}, with an average of removed {mean_value}")

In [ ]:
data = df.groupby(['interview__id','responsible'])[score_name].sum() / df.groupby(['interview__id','responsible'])[score_name].count()
data = data.reset_index()

resp_df = {}
for resp in data['responsible'].unique():
    mask = (data['responsible']==resp)

    total_unit = data[mask]['interview__id'].count()
    mean_value = data[mask][score_name].mean()
    resp_df[resp] = [mean_value]
    print(f"{resp} - Total UNITS: {total_unit}, with an average of removed {mean_value}")

resp_df = pd.DataFrame.from_records(resp_df).T
resp_df = resp_df.reset_index()
resp_df.columns = ['responsible', 'perc']
resp_df.set_index('responsible')['perc'].plot(kind='bar')